In [1]:
from torch.utils.data import DataLoader
from Controller.TrainingParameters import TrainingParameters
from Utils.Constants import Constants
from enum import Enum
import torch
import torch.nn as nn
import torch.optim as optim
from Data.BPI2012Dataset import BPI2012Dataset
from Models.BaselineLSMTModel import BaselineLSTMModel
from Utils.Constants import Constants

/Users/jrhs/.pyenv/versions/dl/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class NotSupportedError(Exception):
    pass

class SelectableDatasets(Enum):
    BPI2012 = 1
    Helpdesk = 2

class SelectableLoss(Enum):
    CrossEntropy = 1

class SelectableModels(Enum):
    BaseLineLSTMModel = 1

class SelectableOptimizer(Enum):
    Adam = 1

class TrainingController:
    def __init__(self, dataset: SelectableDatasets, model: SelectableModels, optimizer: SelectableOptimizer, loss: SelectableLoss):
    
        ## Load dataset
        if (dataset == SelectableDatasets.BPI2012):
            self.dataset = BPI2012Dataset(TrainingParameters.bpi_2012_path)
        else:
            raise NotSupportedError("Dataset you selected is not supported")

        self.data_loader = DataLoader(self.dataset, batch_size=32, shuffle=True, collate_fn= self.dataset.collate_fn)

        # Setting up model 
        if (model == SelectableModels.BaseLineLSTMModel):
            self.model = BaselineLSTMModel(
                vocab_size= self.dataset.vocab_size(),
                embedding_dim= TrainingParameters.BaselineLSTMModelParameters.embedding_dim,
                lstm_hidden= TrainingParameters.BaselineLSTMModelParameters.lstm_hidden,
                dropout= TrainingParameters.BaselineLSTMModelParameters.dropout,
                num_lstm_layers= TrainingParameters.BaselineLSTMModelParameters.num_lstm_layers,
                paddingValue= self.dataset.vocab_to_index(Constants.PAD_VOCAB),
            )
        else:
            raise NotSupportedError("Model you selected is not supported")

        # Setting up optimizer
        if optimizer == SelectableOptimizer.Adam:
            self.opt = optim.Adam(
                self.model.parameters(),
                lr =TrainingParameters.OptimizerParameters.learning_rate,
                                weight_decay= TrainingParameters.OptimizerParameters.l2
                                )
        else: 
             raise NotSupportedError("Optimizer you selected is not supported")

        # Setting up loss

        if (loss == SelectableLoss.CrossEntropy):
            self.loss = nn.CrossEntropyLoss()
        else:
            raise NotSupportedError("Loss function you selected is not supported")
        self.epoch = 0
        self.steps = 0

    def train(self, stop_epoch: int):
        while self.epoch < stop_epoch: 
            self.model.train()
            for i, (_, train, target, lengths) in enumerate(self.data_loader):
                train, target, lengths = train.to(device), target.to(device), lengths.to(device)
                loss, accuracy = self.train_step(train, target, lengths)
                self.steps += 1

                if self.steps % 5 == 0:
                    print('| Epoch [%d] | Step [%d] | lr [%.6f] | Loss: [%.4f] | Acc: [%.4f]|' % (self.epoch, self.steps, self.opt.param_groups[0]['lr'], loss, accuracy))

            self.epoch += 1

    def train_step(self, train, target, lengths):
        self.opt.zero_grad()
        loss, accuracy = self.model_step(train, target, lengths)
        loss.backward()
        self.opt.step()

        return loss, accuracy


    def model_step(self, train, target, lengths):

        out = self.model(train, lengths)

        loss = self.loss(out.transpose(2,1), target)

        accuracy = torch.mean((self.model.get_predicted_seq_from_output(out) == target).float())

        return loss, accuracy

    def reset_epoch(self):
        self.epoch = 0

    def reset_steps(self):
        self.steps = 0

    

In [4]:
trainer = TrainingController(
    dataset= SelectableDatasets.BPI2012, model = SelectableModels.BaseLineLSTMModel, optimizer = SelectableOptimizer.Adam, loss = SelectableLoss.CrossEntropy
)

parsing log, completed traces :: 100%|██████████| 13087/13087 [00:16<00:00, 778.60it/s] 
/Users/jrhs/.pyenv/versions/dl/lib/python3.9/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [5]:
trainer.train(1)

| Epoch [0] | Step [5] | lr [0.005000] | Loss: [3.6699] | Acc: [0.0228]|
| Epoch [0] | Step [10] | lr [0.005000] | Loss: [3.6398] | Acc: [0.0206]|
| Epoch [0] | Step [15] | lr [0.005000] | Loss: [3.5920] | Acc: [0.0312]|
| Epoch [0] | Step [20] | lr [0.005000] | Loss: [3.5496] | Acc: [0.0365]|
| Epoch [0] | Step [25] | lr [0.005000] | Loss: [3.5323] | Acc: [0.0244]|
| Epoch [0] | Step [30] | lr [0.005000] | Loss: [3.5156] | Acc: [0.7324]|
| Epoch [0] | Step [35] | lr [0.005000] | Loss: [3.4747] | Acc: [0.8257]|
| Epoch [0] | Step [40] | lr [0.005000] | Loss: [3.4488] | Acc: [0.7645]|
| Epoch [0] | Step [45] | lr [0.005000] | Loss: [3.4237] | Acc: [0.7504]|
| Epoch [0] | Step [50] | lr [0.005000] | Loss: [3.3967] | Acc: [0.7299]|
| Epoch [0] | Step [55] | lr [0.005000] | Loss: [3.3884] | Acc: [0.7426]|
| Epoch [0] | Step [60] | lr [0.005000] | Loss: [3.3465] | Acc: [0.7915]|
| Epoch [0] | Step [65] | lr [0.005000] | Loss: [3.3384] | Acc: [0.7353]|
| Epoch [0] | Step [70] | lr [0.005000]

In [6]:
b = torch.tensor([[1,1], [1,1]])

In [7]:
torch.mean((a == b).float())

NameError: name 'a' is not defined